# Automatic differentiation

In this notebook, we'll look at an exciting feature of the powerful `tensorflow` library developed by Google and released as an open source library in 2015. Tensorflow has many machine learning functions, but its most prominent use is to set up and train neural networks. Tensorflow works on multiple operating systems, coding languages and can leverage both CPU and GPU capabilities. 

In this exercise we'll look at just one of the features of `tensorflow`, namely _automatic differentiation_. To start with, let's import the library. If this doesn't work it may be because you started a Datascience Environment by mistake.

In [24]:
import tensorflow as tf
import numpy as np

## Python doesn't remember operations

Before we look at automatic differentiation with `tensorflow`, let's recall some basic facts about basic Python. Let's make a variable called `x`, store `5` as a value, and then let `y` be the square of `x`.

In [25]:
x = 5
y = x**2

If we look at what `y` is, we'll just get the value `25`. Nothing more. We can't ask for $\frac{dy}{dx}$ because `y` is just a number that happens to have the value `25`.

In [26]:
y

25

The key feature of `tensorflow` is that it remembers operations, so it can compute the gradient of one variable with respect to another. Let's try that out now.

## Univariate functions with `tensorflow`

Let's try what we did above again with `tensorflow`. The Golden Rule is: for automatic differentiation to work, you **have to do all operations inside tensorflow**. This means that your variables have to be `tensorflow` and, in theory, your operations have to be `tensorflow` operations. An exception to this is that some basic operations like multiply, add etc., are already automatically done with `tensorflow`. 

Let's define our variable `x` again.

In [27]:
x = tf.Variable(5)

This looks okay, but there's an issue. Because we put `5` as the value, `tensorflow` is going to assume this variable is an integer, which will be an issue later when we want to differentiate. Indeed, let's see what happens when we look at `x`.

In [28]:
x

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=5>

See the `dtype=int32`? That's not going to work. To fix this we can either give a `dtype=tf.float32` argument to `tf.Variable`, or we can just make the initial value `5.0` and that will automatically make the number a real number.

In [29]:
x = tf.Variable(5.0)

Tensorflow "records" operations that are executed inside the context of a gradient "tape". This is a little bit complicated to explain fully, but what matters is the coding syntax we use. Let's suppose we want to let `y` be `x` squared, and we want `tensorflow` to track that operation. We do this as follows.

In [30]:
with tf.GradientTape() as tape:
    y = x**2

Let's see what's in `y`.

In [31]:
y

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

It has a value of `25` as you can see. But because we tracked the operation, we can now calculate the value of $\frac{dy}{dx}$ at $x = 5$ with a single call.

In [32]:
dy_dx = tape.gradient(y,x)

In [33]:
dy_dx

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

Note that we used `tape` to indicate how the operations were recorded. We got the value of `dy_dx` as $10$. Check for yourself that this is the correct value of the derivative of the function $y = x^2$ at $x = 5$. What's amazing is that `tensorflow` knows the Chain Rule, so you can use it to calculate the gradient for complicated functions. Let's do this now, we'll calculate the derivative of the function 
$$
f(x) = \ln(2(x+1)^3+5x)
$$
at the value $x = 3$. 

In [34]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
    y = tf.math.log(2*(x+1)**3+5*x)

Notice how we didn't use the `log` function in `numpy` or anywhere else. We have to use the `tensorflow` version for this to work.

In [35]:
dy_dx = tape.gradient(y,x)
dy_dx

<tf.Tensor: shape=(), dtype=float32, numpy=0.7062937>

You can check this by hand or using WolframAlpha if you like. One technical thing to note is that if you try and run the code block above again (feel free to try it), you'll get an error. As soon as you compute a gradient using a tape, `tensorflow` assumes you're done with that tape and discards it to save memory. This makes sense because `tensorflow` is designed for large scale machine learning computations. You can force it to stick around, though, using a `persistent=True` flag when making the tape.

We could also have built up the function $f$ piece by piece and let `tensorflow` do the Chain Rule for us. Hopefully we got the same answer. 

In [36]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
    y1 = 2*(x+1)**3 + 5*x
    y2 = tf.math.log(y1)
dy_dx = tape.gradient(y2,x)
dy_dx 

<tf.Tensor: shape=(), dtype=float32, numpy=0.7062937>

## Gradients with `tensorflow`

Tensorflow loves to work with vectors (and even tensors!), so computing gradients is also simple. The main pitfall to avoid is using `numpy` operations instead of `tensorflow` ones. Let's compute the gradient of the function $f: \mathbb{R}^3 \to \mathbb{R},\ f(\mathbf{x}) = ||x||^2$ at $\mathbf{x} = [1,1,1]^\mathsf{T}$. We first create a variable as before, but this time it's a vector.

In [37]:
x = tf.Variable([1.0,1.0,1.0])

In [38]:
x

<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>

Let's set up a tape and calculate the norm of `x`. Remember to use `tensorflow` operations! If you not sure what the right function is in tensorflow, just Google it (that's what I did!).

In [39]:
with tf.GradientTape() as tape:
    y = tf.norm(x)**2

In [40]:
dy_dx = tape.gradient(y,x)
dy_dx 

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>

We should get an answer of $2x^\mathsf{T}$, and indeed that's what we get (though `tensorflow` doesn't distinguish between row and column vectors here).

Let's check that the chain rule still works but computing the gradient of the function $f(\mathbf{x}) = ||x||^4$.

In [41]:
x = tf.Variable([1.0,1.0,1.0])
with tf.GradientTape() as tape:
    y1 = tf.norm(x)
    y2 = y1**4
dy_dx = tape.gradient(y2,x)
dy_dx 

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([12., 12., 12.], dtype=float32)>

You'll have to finish the Theory Homework to check this one. Finally, note that if you accidentally ask for the gradient of a vector-valued function $f$, `tensorflow` will just take the gradient of each function $f_i$ and add them. When you want a Jacobian, you should use a different function, which we look at next.

## Jacobians with `tensorflow`

Let's compute the Jacobian of the function $f: \mathbb{R}^3 \to \mathbb{R}^2$ given by 
$$
f(\mathbf{x}) = A\mathbf{x} 
$$
where 
$$
A = \begin{pmatrix} 1 & -1 & 2 \\ 5 & 2 & 3\end{pmatrix}
$$

If you recall what we did in class, you should get $A$ itself as the Jacobian, no matter what value $\mathbf{x}$ takes. With that in mind, let's make $\mathbf{x}$ a completely random vector. We want it to be of size 3, so we give `tf.random.uniform` a single argument of `[3]` to tell it what shape to give us. 

The matrix `A` is not a variable, it's fixed, so we define it as a `tf.constant`. And we remember to use `tensorflow` operations for the matrix-vector product.

In [42]:
x = tf.Variable(tf.random.uniform([3]))
A = tf.constant([[1.0,-1.0,2.0], [5.0,2.0,3.0]])
with tf.GradientTape() as tape:
    y = tf.linalg.matvec(A, x)

Let's see what we got using `tape.jacobian` instead of `gradient`.

In [43]:
dy_dx = tape.jacobian(y,x)
dy_dx 

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1., -1.,  2.],
       [ 5.,  2.,  3.]], dtype=float32)>

We have barely scratched the surface of automatic differentiation in `tensorflow`. In practice, a lot of this happens in the background when running large neural networks, but it's worth exploring on its own. You don't really need to know the actual base-level implementation of these operations to use them.

## Exercise 1

Use `tensorflow` to find the derivative of the function $y = e^{(x-1)^2}$ at $x = 4$. 

In [44]:
tf.math.pow(2,2) #testing this to make sure tf.math.pow is the right thing to use

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [45]:
x = tf.Variable(4.0)
with tf.GradientTape() as tape:
    y = tf.math.exp(tf.math.pow(x-1, 2))
dy_dx = tape.gradient(y,x)
dy_dx

<tf.Tensor: shape=(), dtype=float32, numpy=48618.504>

## Exercise 2

Use `tensorflow` to find the gradient of the function $f: \mathbb{R}^2 \to \mathbb{R},\ f(x,y) = (x-1)^2 - y^2+1$ at $(x,y) = (2,3)$.

In [57]:
x , y = tf.Variable(2.0) , tf.Variable(3.0)

In [60]:
with tf.GradientTape() as tape: 
    z = tf.math.pow(x-1, 2) - tf.math.pow(y, 2) + 1

In [61]:
gradients = tape.gradient(z , [x , y])
print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, <tf.Tensor: shape=(), dtype=float32, numpy=-6.0>]
